<a href="https://colab.research.google.com/github/windyday0622/windyday/blob/main/m3_%EB%B6%84%EC%84%9D%20%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC/%20pandas_%EC%8B%A4%EC%8A%B5_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[실습] titanic.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving titanic3.csv to titanic3 (1).csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

file_path = 'titanic3.csv'
df = pd.read_csv(file_path)

print(df.head())

   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

     age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
0  29.00      0      0   24160  211.3375       B5        S    2    NaN   
1   0.92      1      2  113781  151.5500  C22 C26        S   11    NaN   
2   2.00      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
3  30.00      1      2  113781  151.5500  C22 C26        S  NaN  135.0   
4  25.00      1      2  113781  151.5500  C22 C26        S  NaN    NaN   

                         home.dest  
0                     St Louis, MO  


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [ ]:
# 결측치 확인
print(df.isnull().sum())

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


In [ ]:
# 필요없는 열 제거
df.drop(columns=['name', 'ticket', 'body', 'home.dest', 'boat', 'pclass'], inplace=True)
df.head()

,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,female,29.00,0,0,211.3375,B5,S
1,1,male,0.92,1,2,151.5500,C22 C26,S
2,0,female,2.00,1,2,151.5500,C22 C26,S
3,0,male,30.00,1,2,151.5500,C22 C26,S
4,0,female,25.00,1,2,151.5500,C22 C26,S


In [ ]:
# 결측치 처리
import pandas as pd

# 각 열을 수정하려면 다음과 같이 할당을 올바르게 해야 합니다:
df['age'] = df['age'].fillna(df['age'].mean(), inplace=False)
df['fare'] = df['fare'].fillna(df['fare'].mean(), inplace=False)
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[[0]], inplace=False)

# 'cabin' 열 삭제
df.drop(columns=['cabin'], inplace=True)

In [ ]:
bins = [0, 5, 12, 18, 35, 60, 100]
labels = ['Infant', 'Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)
df.drop(columns=['age'], inplace=True)

In [ ]:
fare_bins = [0, 30, 100, 600]
fare_labels = ['Low', 'Medium', 'High']
df['fare_group'] = pd.cut(df['fare'], bins=fare_bins, labels=fare_labels)
df.drop(columns=['fare'], inplace=True)

In [ ]:
df['family_size'] = df['sibsp']+ df['parch'] + 1
df.drop(columns=['sibsp', 'parch'], inplace=True)

In [ ]:
categorical_columns = ['age_group', 'fare_group', 'sex', 'embarked']

for column in categorical_columns:
    df = pd.get_dummies(df, columns=[column], drop_first=True)

In [ ]:
df.head()

,survived,family_size,age_group_Child,age_group_Teenager,age_group_Young Adult,age_group_Adult,age_group_Senior,fare_group_Medium,fare_group_High,sex_male,embarked_Q,embarked_S
0,1,1,False,False,True,False,False,False,True,False,False,True
1,1,4,False,False,False,False,False,False,True,True,False,True
2,0,4,False,False,False,False,False,False,True,False,False,True
3,0,4,False,False,True,False,False,False,True,True,False,True
4,0,4,False,False,True,False,False,False,True,False,False,True


In [ ]:
# 타겟 변수와 피처 변수 설정
X = df.drop('survived', axis=1)
y = df['survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

(1047, 11) (262, 11) (1047,) (262,)
Accuracy: 0.7633587786259542
Confusion Matrix:
 [[123  21]
 [ 41  77]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.85      0.80       144
           1       0.79      0.65      0.71       118

    accuracy                           0.76       262
   macro avg       0.77      0.75      0.76       262
weighted avg       0.77      0.76      0.76       262



[실습] train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

Survived: 생존 여부 (목표 변수).
Pclass: 티켓 등급.
Sex: 승객의 성별 (0 = 남성, 1 = 여성).
Age: 승객의 나이.
SibSp: 타이타닉에 탑승한 형제/배우자의 수.
Parch: 타이타닉에 탑승한 부모/자녀의 수.
Fare: 승객 요금.
Embarked_Q: 퀸즈타운에서 탑승했는지 여부 (1 = 예, 0 = 아니오).
Embarked_S: 사우샘프턴에서 탑승했는지 여부 (1 = 예, 0 = 아니오).

데이터 분할: 데이터 세트를 훈련 데이터와 테스트 데이터로 나눈다.
모델 선택 및 훈련: 예측 모델을 선택하고 훈련 데이터를 사용하여 모델을 훈련.
모델 평가: 테스트 데이터를 사용하여 모델의 성능을 평가.
모델 개선: 필요에 따라 모델을 개선.

1. 데이터 분할
2. 결측치 확인 및 대체
3. 랜덤 포레스트 모델 학습
4. 예측 및 모델 평가
정확도(accuracy)를 계산.

In [109]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (3).csv


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [111]:
# 필요한 라이브러리 가져오기
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [112]:
# 데이터 로드 및 준비

train_data = pd.read_csv('train.csv')
X = train_data.drop(columns=['Survived'])
y = train_data['Survived']

In [113]:
# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [114]:
# 결측치 확인
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [117]:
# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 11) (179, 11) (712,) (179,)


In [118]:
# 데이터셋에서 수치형과 범주형 데이터를 구분
numeric_cols = X_train.select_dtypes(include=['number']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# 결측값 대체
numeric_imputer = SimpleImputer(strategy='mean')
X_train_numeric_imputed = numeric_imputer.fit_transform(X_train[numeric_cols])
X_test_numeric_imputed = numeric_imputer.transform(X_test[numeric_cols])

# 범주형 데이터의 결측값 대체
mode_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical_imputed = mode_imputer.fit_transform(X_train[categorical_cols])
X_test_categorical_imputed = mode_imputer.transform(X_test[categorical_cols])

# 범주형 데이터에 대해 원-핫 인코딩 수행
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_encoded = encoder.fit_transform(X_train_categorical_imputed)
X_test_categorical_encoded = encoder.transform(X_test_categorical_imputed)

# 데이터프레임으로 변환
X_train_numeric_imputed = pd.DataFrame(X_train_numeric_imputed, columns=numeric_cols)
X_test_numeric_imputed = pd.DataFrame(X_test_numeric_imputed, columns=numeric_cols)
X_train_categorical_encoded_df = pd.DataFrame(X_train_categorical_encoded.toarray(), columns=encoder.get_feature_names_out(input_features=categorical_cols))
X_test_categorical_encoded_df = pd.DataFrame(X_test_categorical_encoded.toarray(), columns=encoder.get_feature_names_out(input_features=categorical_cols))

# 결측값 대체된 수치형 데이터와 범주형 데이터를 다시 결합
X_train_imputed = pd.concat([X_train_numeric_imputed, X_train_categorical_encoded_df], axis=1)
X_test_imputed = pd.concat([X_test_numeric_imputed, X_test_categorical_encoded_df], axis=1)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_imputed, y_train)

# 예측
y_pred = model.predict(X_test_imputed)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8044692737430168
Confusion Matrix:
 [[93 12]
 [23 51]]
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.89      0.84       105
           1       0.81      0.69      0.74        74

    accuracy                           0.80       179
   macro avg       0.81      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [ ]:
# 수치형과 범주형 데이터 구분: 데이터프레임을 통해 수치형과 범주형 데이터를 구분합니다.
# 결측값 대체: 수치형 데이터는 평균값으로, 범주형 데이터는 최빈값으로 결측값을 대체합니다.
# 범주형 데이터 원-핫 인코딩: 범주형 데이터를 원-핫 인코딩하여 숫자형으로 변환합니다.
# 데이터프레임으로 변환: 인코딩된 범주형 데이터를 데이터프레임으로 변환합니다.
# 결합: 결측값 대체된 수치형 데이터와 범주형 데이터를 다시 결합하여 전체 특성 행렬을 구성합니다.
# 랜덤 포레스트 모델 학습: RandomForestClassifier를 사용하여 모델을 학습시킵니다.
# 예측: 학습된 모델을 사용하여 테스트 데이터에 대해 예측을 수행합니다.
# 성능 평가: 예측 결과를 실제값과 비교하여 정확도, 오차 행렬, 분류 보고서를 출력합니다.